<a href="https://colab.research.google.com/github/Juan-Draghi/monitoreo_web/blob/main/Monitoreo_web_v_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Script para monitorear cambios en sitios web (versión 2)**
**¿Qué hace esta versión?**

1. Nueva carpeta para hashes: Guarda los hashes en una carpeta diferente en Google Drive (web_monitoring_hashes_text) para distinguirlos de los hashes del HTML completo.
2. Función extract_visible_text: Utiliza BeautifulSoup para obtener solo el texto visible de la página, eliminando todas las etiquetas HTML.
3. Comparación basada en el texto: El hash ahora se calcula sobre este texto visible, y la comparación se realiza entre los hashes del texto.

In [ ]:
import requests
import hashlib
import os
from google.colab import drive
from bs4 import BeautifulSoup

# --- Configuración ---
urls_a_monitorear = [
    "https://buenosaires.gob.ar/jefaturadegabinete/desarrollo-urbano/normativa/codigo-urbanistico-y-de-edificacion",
    "https://buenosaires.gob.ar/jefaturadegabinete/desarrollo-urbano/novedades-de-la-subsecretaria-de-gestion-urbana",
    "https://buenosaires.gob.ar/agc/noticias",
    "https://www.cpau.org/noticias",
    "https://buenosaires.gob.ar/noticias/desarrollo-urbano",
    "https://adrianmercadorealestate.com/blog/informes",
    "https://www.estadisticaciudad.gob.ar/eyc/?page_id=1479",
    "https://www.ceso.com.ar/secciones/provincias-y-regiones",
    "https://colegioinmobiliario.org.ar/institucional/observatorioEstadistico",
    "https://colegioinmobiliario.org.ar/novedades",
    "https://www.colliers.com/es-ar",
    "https://www.afcp.org.ar/despacho-mensual",
    "https://www.indec.gob.ar/indec/web/Nivel3-Tema-3-3",
    "https://www.remax.com.ar/indice-remax",
    "https://www.ieric.org.ar/estadistica/informes-de-coyuntura/?2025",
    "https://www.uade.edu.ar/sites/investigacion/instituto-de-economia-ineco/informes-y-novedades/mei-informe-del-mercado-inmobiliario-e-indice-del-salario-real/",
    "https://www.nmrk.com.ar/informe/11023/reportes-de-mercado-2025",
    "https://www.fabianachaval.com/blog",
    "https://sites.google.com/view/red-operaciones-inmobiliarias",
    "https://www.ljramos.com.ar/informes-del-mercado-inmobiliario",
    "https://www.zonaprop.com.ar/noticias/zpindex/",
    "https://www.zonaprop.com.ar/noticias/zpindex/gba-venta/",
    "https://www.zonaprop.com.ar/noticias/zpindex/gba-oeste-sur-venta/",
    "https://colegioinmobiliario.org.ar/novedades",
    "https://www.cpau.org/noticias",
    "https://noticias.argenprop.com/"
]
drive_base_path = '/content/drive/MyDrive/web_monitoring_hashes_text' # Nueva carpeta para hashes de texto

# --- Funciones ---

def get_page_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': url
    }
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener {url}: {e}")
        return None

def extract_visible_text(html_content):
    if not html_content:
        return ""
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        # Obtener todo el texto visible, separando con espacios y eliminando espacios extra
        text = soup.get_text(separator=' ', strip=True)
        return text
    except Exception as e:
        print(f"Error al extraer texto: {e}")
        return ""

def calculate_hash(text):
    if not text:
        return "d41d8cd98f00b204e9800998ecf8427e"
    return hashlib.sha256(text.encode('utf-8', errors='ignore')).hexdigest()

def get_filename_from_url(url):
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest()
    return f"{url_hash}_text.txt" # Sufijo para distinguir de hashes de HTML

def save_last_hash(file_path, current_hash):
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            f.write(current_hash)
    except Exception as e:
        print(f"Error al guardar hash en {file_path}: {e}")

def load_last_hash(file_path):
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r') as f:
                return f.read().strip()
        except Exception as e:
            print(f"Error al cargar hash desde {file_path}: {e}")
            return None
    return None

# --- Ejecución principal ---

print("Montando Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado.")
    drive_mounted = True
except Exception as e:
    print(f"Error al montar Google Drive: {e}")
    print("No se podrá guardar/cargar el estado anterior.")
    drive_mounted = False

print(f"\nIniciando monitoreo de {len(urls_a_monitorear)} URL(s) (comparando texto visible)...")
cambios_detectados = []
urls_con_error = []

for url in urls_a_monitorear:
    print(f"\n--- Procesando: {url} ---")
    html_content = get_page_content(url)

    if html_content is None:
        print(f"No se pudo obtener contenido de {url}. Saltando.")
        urls_con_error.append(url)
        continue

    visible_text = extract_visible_text(html_content)
    current_hash = calculate_hash(visible_text)

    if not current_hash:
        print(f"No se pudo calcular el hash del texto de {url}. Saltando.")
        urls_con_error.append(url)
        continue

    if drive_mounted:
        url_state_filename = get_filename_from_url(url)
        url_state_path = os.path.join(drive_base_path, url_state_filename)
        last_hash = load_last_hash(url_state_path)

        if last_hash is None:
            print("  Estado anterior no encontrado. Guardando hash del texto actual.")
            save_last_hash(url_state_path, current_hash)
        elif current_hash != last_hash:
            print("  ¡CAMBIO DETECTADO en el texto visible!")
            cambios_detectados.append(url)
            save_last_hash(url_state_path, current_hash)
        else:
            print("  No se detectaron cambios en el texto visible.")
    else:
        print("  Google Drive no está montado. No se puede comparar el estado anterior.")

print("\n--- Resumen del Monitoreo ---")
if cambios_detectados:
    print(f"Se detectaron cambios en el texto visible de las siguientes {len(cambios_detectados)} URL(s):")
    for url_cambiada in cambios_detectados:
        print(f"- {url_cambiada}")
else:
    print("No se detectaron cambios en el texto visible de las URLs procesadas.")

if urls_con_error:
    print(f"\nOcurrieron errores al procesar las siguientes {len(urls_con_error)} URL(s):")
    for url_error in urls_con_error:
        print(f"- {url_error}")

# Desmontar Drive al finalizar (opcional)
# drive.flush_and_unmount()